# Project 1: Personalized Programming Tutor

## Executive Summary

This project demonstrates the creation of an **AI-powered educational tool** designed to explain complex programming concepts in a personalized, easy-to-understand manner. 

By leveraging Large Language Models (LLMs), this application acts as a virtual tutor that:
1.  **Analyzes** raw code snippets provided by the user.
2.  **Deconstructs** the logic into step-by-step explanations.
3.  **Teaches** the underlying concepts and best practices.
4.  **Delivers** content using real-time streaming for an engaging user experience.

**Key Engineering Concepts:**
- **LLM Integration**: Seamless switching between OpenAI (Cloud) and Ollama (Local) providers.
- **Prompt Engineering**: Designing robust system prompts to enforce a specific teaching persona.
- **Streaming Responses**: Implementing token-by-token rendering for low-latency feedback.
- **Interactive UI**: Building a command-line interface (CLI) for user interaction.

## 1. Environment Setup & Imports

We begin by importing the necessary libraries. 
- `os` and `dotenv`: For secure management of API keys and configuration via environment variables.
- `IPython.display`: To render Markdown and handle real-time display updates in the notebook.
- `openai`: The standard client library used to interact with both OpenAI's API and Ollama's OpenAI-compatible endpoint.

In [ ]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

## 2. Provider Configuration

This system is designed to be **model-agnostic**. You can switch between a cloud provider (OpenAI) for maximum capability or a local provider (Ollama) for privacy and zero cost.

Change the `USE_PROVIDER` variable below to switch backends.

In [ ]:
# CONFIGURATION: Choose your LLM provider
# Options: 'ollama' (Local) or 'openai' (Cloud)
USE_PROVIDER = 'ollama' 

print(f"Selected provider: {USE_PROVIDER.upper()}")

## 3. Client Initialization

Here we initialize the API client based on the selected provider. 

- **Ollama**: Connects to a local server (usually `localhost:11434`). It mimics the OpenAI API structure, allowing us to use the same client code.
- **OpenAI**: Connects to the official API using your secret key.

This abstraction layer allows the rest of the application to remain unchanged regardless of the underlying model.

In [ ]:
# Load environment variables from .env file
load_dotenv(dotenv_path='/workspace/.env', override=True)

# Determine provider from code or environment
llm_provider = USE_PROVIDER if 'USE_PROVIDER' in globals() else os.getenv('LLM_PROVIDER', 'ollama')

if llm_provider == 'ollama':
    # --- OLLAMA CONFIGURATION (Local) ---
    ollama_base_url = os.getenv('OLLAMA_BASE_URL', 'http://localhost:11434')
    ollama_api_key = os.getenv('OLLAMA_API_KEY', 'ollama')
    ollama_model = 'qwen3-coder:480b-cloud' # Example high-performance local model
    
    print("=" * 50)
    print("🚀 Using OLLAMA (Local)")
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print("=" * 50)
    
    # Initialize OpenAI client pointing to local Ollama instance
    client = OpenAI(
        base_url=f"{ollama_base_url}/v1",
        api_key=ollama_api_key
    )
    MODEL = ollama_model
    
else:
    # --- OPENAI CONFIGURATION (Cloud) ---
    api_key = os.getenv('OPENAI_API_KEY')
    
    print("=" * 50)
    print("☁️ Using OPENAI (Cloud API)")
    print(f"   Model: gpt-4o-mini")
    print("=" * 50)
    
    client = OpenAI(api_key=api_key)
    MODEL = 'gpt-4o-mini'

## 4. The Inference Engine

This function, `callModel`, is the core of our application. It handles the communication with the LLM.

**Key Feature: Streaming**
When `use_stream=True`, the function yields chunks of text as they are generated. We use `IPython.display.update_display` to append these chunks to the screen in real-time, creating a "typewriter" effect. This significantly improves perceived performance, as the user doesn't have to wait for the entire explanation to be generated.

In [ ]:
def callModel(messages, use_stream=False):
    """
    Sends messages to the LLM and handles the response.
    
    Args:
        messages (list): List of message dictionaries (role, content).
        use_stream (bool): Whether to stream the response in real-time.
    """
    if use_stream:
        stream = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            stream=True
        )
        
        response_text = ""
        display_handle = None
        
        for chunk in stream:
            content = chunk.choices[0].delta.content or ''
            response_text += content
            
            # Update the display dynamically
            if display_handle is None and response_text:
                display_handle = display(Markdown(response_text), display_id=True)
            elif display_handle is not None:
                update_display(Markdown(response_text), display_id=display_handle.display_id)
        
    else:
        # Standard non-streaming call
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        result = response.choices[0].message.content
        display(Markdown(result))

## 5. Persona Design (System Prompt)

The **System Prompt** is where we define the AI's behavior. For this project, we are crafting a "Personalized Tutor" persona.

We explicitly instruct the model to:
- Avoid overly technical jargon without explanation.
- Use analogies to make abstract concepts concrete.
- Focus on the "why" and "how", not just the syntax.
- Format the output cleanly with Markdown.

In [ ]:
system_prompt = """
You are an expert programming tutor with deep knowledge in Python, JavaScript, and software engineering.

Your teaching style:
- Explain concepts clearly and step by step.
- Use analogies and real-world examples to make concepts stick.
- Break down complex code into understandable parts.
- Highlight best practices and common pitfalls.
- Encourage learning by explaining the "why" behind the code.

Response format:
- Always respond in well-formatted Markdown.
- Use headers, lists, and code blocks appropriately.
- Do NOT wrap your entire response in markdown code fences (```).
- Be concise but thorough.
"""

## 6. User Interaction

We create a simple interactive prompt to accept code snippets from the user. This simulates a real-world scenario where a student asks for help with a specific block of code.

In [ ]:
print("=" * 60)
print("🎓 PERSONALIZED PROGRAMMING TUTOR")
print("=" * 60)
print("\nEnter the code you want to understand.")
print("\nExample:")
print('  yield from {book.get("author") for book in books if book.get("author")}')
print("\nYour question (paste your code below):")
print("-" * 60)

question = input()

## 7. Prompt Engineering

We wrap the user's raw input into a structured prompt. This technique ensures the model knows exactly *what* to do with the code (explain it) rather than just executing it or refactoring it.

We specifically ask for:
1. Functionality summary.
2. Step-by-step breakdown.
3. Use cases.
4. Key concepts.

In [ ]:
user_prompt = f"""
Please explain the following code in detail:

```python
{question}
```

Include:
1. What this code does
2. How it works step by step
3. Why someone would use this approach
4. Any important concepts or patterns involved
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

## 8. Execution

Finally, we call the model with our constructed messages and enable streaming to see the tutor in action.

In [ ]:
# Execute the tutor with the question
callModel(messages, use_stream=True)

## Conclusion

You have successfully built a **Personalized Programming Tutor**. 

This project demonstrates how to:
- Configure and use different LLM providers.
- Design effective system prompts to create specific AI personas.
- Implement streaming for better user experience.
- Structure a clean, professional AI application.

**Future Improvements:**
- Add a "Follow-up Question" feature to make it a conversation.
- Integrate syntax highlighting for the code blocks in the output.
- Add support for uploading code files directly.